In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set a rsandom seed for reproducibility
torch.manual_seed(42)

In [3]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
# train test split

x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [5]:
X_train , X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [6]:
#scaling the features

X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
# Create a CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, Features, Labels):
        self.Features = torch.tensor(Features, dtype=torch.float32)
        self.Labels = torch.tensor(Labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.Features)
    
    def __getitem__(self, idx):
        return self.Features[idx], self.Labels[idx]


In [8]:
# create a train_dataset object 
train_dataset = CustomDataset(X_train, y_train)

In [9]:
len(train_dataset)

4800

In [10]:
# create test_dataset object 
test_dataset = CustomDataset(X_test, y_test)

In [11]:
len(test_dataset)

1200

In [12]:
# create and test loader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
# Define the NN class

class MyNN(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)

In [19]:
# set learning rate and epochs
lr = 0.1
epochs = 100

In [35]:
# Instantiate the model
model = MyNN(X_train.shape[1])


# loss function 
criterion = nn.CrossEntropyLoss()

# using the best optimizer for the model - adam / sgd / rmsprop
optimizer = optim.SGD(model.parameters(), lr=lr)

In [36]:
# training loop 

for epoch in range(epochs):
    output = model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 0.7496231198310852
Epoch 2/100, Loss: 0.6229442358016968
Epoch 3/100, Loss: 0.8218339085578918
Epoch 4/100, Loss: 0.4268905818462372
Epoch 5/100, Loss: 0.7349354028701782
Epoch 6/100, Loss: 0.4638618230819702
Epoch 7/100, Loss: 0.35462889075279236
Epoch 8/100, Loss: 0.3960343301296234
Epoch 9/100, Loss: 0.6211223602294922
Epoch 10/100, Loss: 0.20655426383018494
Epoch 11/100, Loss: 0.5043542981147766
Epoch 12/100, Loss: 0.42317312955856323
Epoch 13/100, Loss: 0.2955767512321472
Epoch 14/100, Loss: 0.6730344295501709
Epoch 15/100, Loss: 0.34224578738212585
Epoch 16/100, Loss: 0.3603803217411041
Epoch 17/100, Loss: 0.4556501507759094
Epoch 18/100, Loss: 0.22816652059555054
Epoch 19/100, Loss: 0.2377115935087204
Epoch 20/100, Loss: 0.3410474956035614
Epoch 21/100, Loss: 0.2577414810657501
Epoch 22/100, Loss: 0.33742132782936096
Epoch 23/100, Loss: 0.23675477504730225
Epoch 24/100, Loss: 0.21209625899791718
Epoch 25/100, Loss: 0.11783583462238312
Epoch 26/100, Loss: 0.238

In [37]:
# set the model to evaluation mode
model.eval()


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [38]:
# evaluate the model
total = 0
correct = 0

with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [39]:
# print the accuracy
print(f'Accuracy: {correct/total}')

Accuracy: 0.8425
